In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///private/var/folders/c_/qfmhj66j0tn016nkx_th4hxm0000gp/T/abs_4bu2zqzzh9/croot/scikit-learn_1714164755228/work


In [2]:
!python -V

Python 3.12.4


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/levan/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/levan/anaconda3/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']


def read_data(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')

    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [10]:
deviation = abs(df['duration'] - y_pred)
mean_deviation = deviation.mean()
print(f"Q1. Notebook. standard deviation: {mean_deviation}")

Q1. Notebook. standard deviation: 6.249106352794334


In [18]:
# Define the year and month
year = 2024
month = 3

# Create the 'ride_id' column by concatenating the year/month and index
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype(str)

# Display the first few rows to verify the new column
df[['ride_id']].head()


,ride_id


In [19]:
# Save the updated DataFrame if needed
df.to_parquet('yellow_tripdata_2024-03_with_ride_id.parquet')
